## TensorFlow の読み込み

In [1]:
# パッケージの読み込み
import tensorflow as tf

# バージョン確認
tf.__version__

'1.10.1'

## 足し算のグラフを演算

In [2]:
with tf.Graph().as_default():
  # 定数の定義
  x = tf.constant(10, name="x")
  y = tf.constant(32, name="y")

  # 足し算
  op = tf.add(x, y)

  # 演算を実行
  with tf.Session() as sess:
    result = sess.run(op)

print(result)  # -> 42

42


## 入力を受け付けるグラフ

In [3]:
with tf.Graph().as_default():
  # xは入力を受け付ける入れ物
  x = tf.placeholder(tf.int32, name="x")
  y = tf.constant(32, name="y")

  # 足し算
  op1 = tf.add(x, y)

  # 足し算結果を格納する変数
  z = tf.Variable(0, name="z")

  # 足し算結果を変数に代入
  op2 = tf.assign(z, op1)

  # 変数の初期化
  init_op = tf.global_variables_initializer()

  # 演算を実行
  with tf.Session() as sess:
    # 変数の初期化を実行
    sess.run(init_op)
    # xに10をfeed_dictで入力し、結果を得る
    result = sess.run(op2, feed_dict={x: 10})

print(result)

42


# tf.layers

## 乳がんのデータをロード

In [0]:
from sklearn.datasets import load_breast_cancer

# 乳がんのデータをロード
X_dataset, y_dataset = load_breast_cancer(return_X_y=True)

## 学習用とテスト用に分け、スケーリングする

In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# X_datasetをX_trainとX_testに
# y_datasetをy_trainとy_testに分割
X_train, X_test, y_train, y_test = train_test_split(
    X_dataset, y_dataset, test_size=0.2, random_state=42)

# データを0~1の範囲にスケール
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## tf.layers を使ったニューラルネットワークで学習、正解率を算出

In [6]:
import tensorflow as tf

NUM_FEATURES = 30
NUM_UNITS_H1 = 4
NUM_UNITS_H2 = 4
NUM_CLASSES = 2

with tf.Graph().as_default():
  # 入力層
  X = tf.placeholder(tf.float32, shape=[None, NUM_FEATURES], name="X")
  y = tf.placeholder(tf.float32, shape=[None, ], name="y")

  # 隠れ層
  hidden1 = tf.layers.dense(inputs=X,
                            units=NUM_UNITS_H1,
                            activation=tf.nn.relu,
                            name='hidden1')
  hidden2 = tf.layers.dense(inputs=hidden1,
                            units=NUM_UNITS_H2,
                            activation=tf.nn.relu,
                            name='hidden2')

  # 出力層
  logits = tf.layers.dense(inputs=hidden2,
                           units=NUM_CLASSES,
                           name='output')

  # 損失
  onehot_labels = tf.one_hot(indices=tf.cast(y, tf.int32),
                             depth=NUM_CLASSES)
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
      labels=onehot_labels,
      logits=logits,
      name='xentropy')
  loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

  # 損失を最小化
  train_op = tf.train.AdamOptimizer(0.01).minimize(loss)

  # テスト用の正解率演算オペレーション
  correct_prediction = tf.equal(tf.argmax(logits, 1),
                                tf.argmax(onehot_labels, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(1000):
      _, loss_value = sess.run([train_op, loss],
                               feed_dict={X: X_train, y: y_train})
      if step % 100 == 0:
        print('Step: %d, Loss: %f' % (step, loss_value))

    # テストデータで正解率を求める
    acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
    print('Accuracy: %f' % acc)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Step: 0, Loss: 0.682066
Step: 100, Loss: 0.053717
Step: 200, Loss: 0.030405
Step: 300, Loss: 0.022486
Step: 400, Loss: 0.016756
Step: 500, Loss: 0.010345
Step: 600, Loss: 0.007275
Step: 700, Loss: 0.005580
Step: 800, Loss: 0.004672
Step: 900, Loss: 0.003965
Accuracy: 0.973684
